In [ ]:
from player import Player, Game

In [ ]:
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import socceraction.spadl.config as spadlconfig
from socceraction.spadl.schema import SPADLSchema
from pandera.typing import DataFrame, Series
import numpy.typing as npt
from sklearn.exceptions import NotFittedError
from typing import Callable, List, Optional, Tuple
import os
import copy

In [ ]:
xTModell = xthreat.load_model("/home/morten/Develop/packing-report/xT-impact/models/xT_full_data")

In [ ]:
def get_top_league_elo(df_elo):
    all_leagues = ["GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", "FRA-Ligue 1", "ITA-Serie A"]
    top_league = 0
    for league in all_leagues:
        league_elo = np.mean(df_elo[df_elo['league'] == league]['elo'])
        if league_elo > top_league:
            top_league = league_elo
    return top_league

def up_team_count(count_t, df_g, g_id):
    h_id = df_g[df_g['game_id'] == g_id]['home_team_id'].values[0]
    a_id = df_g[df_g['game_id'] == g_id]['away_team_id'].values[0]

    if h_id in count_t['all']:
        count_t['all'][h_id] += 1
    else:  
        count_t['all'][h_id] = 0
    if a_id in count_t['all']:
        count_t['all'][a_id] += 1
    else:  
        count_t['all'][a_id] = 0

    if h_id in count_t['home']:
        count_t['home'][h_id] += 1
    else:  
        count_t['home'][h_id] = 0
    if a_id in count_t['away']:
        count_t['away'][a_id] += 1
    else:  
        count_t['away'][a_id] = 0

def calc_form(form_list):
    form = 0
    for f in form_list:
        form += int(f)
    return form

def calc_table_pos(c_teams, t_id, o_id, home):
    table_dict = {}
    game_day = c_teams['all'][t_id]
    game_day_t = c_teams['home'][t_id] if home else c_teams['away'][t_id]
    game_day_o = c_teams['away'][o_id] if home else c_teams['home'][o_id]
    table_dict['team_table_pos'] = tables['all'][game_day][tables["all"][game_day]['team_id'] == t_id].index[0] + 1
    table_dict['opp_table_pos'] = tables['all'][game_day][tables["all"][game_day]['team_id'] == o_id].index[0] + 1
    table_dict['team_home_away_table_pos'] = (tables['home'][game_day_t][tables["home"][game_day_t]['team_id'] == t_id].index[0] + 1) if home else (tables['away'][game_day_t][tables["away"][game_day_t]['team_id'] == t_id].index[0] + 1)
    table_dict['opp_home_away_table_pos'] = (tables['away'][game_day_o][tables["away"][game_day_o]['team_id'] == o_id].index[0] + 1) if home else (tables['home'][game_day_o][tables["home"][game_day_o]['team_id'] == o_id].index[0] + 1)
    
    table_dict['team_form_for'] = calc_form(tables['all'][game_day][tables["all"][game_day]['team_id'] == t_id]['form_for'].values[0])
    table_dict['team_form_against'] = calc_form(tables['all'][game_day][tables["all"][game_day]['team_id'] == t_id]['form_against'].values[0])
    table_dict['opp_form_for'] = calc_form(tables['all'][game_day][tables["all"][game_day]['team_id'] == o_id]['form_for'].values[0])
    table_dict['opp_form_against'] = calc_form(tables['all'][game_day][tables["all"][game_day]['team_id'] == o_id]['form_against'].values[0])
    
    table_dict['team_home_away_form_for'] = calc_form(tables['home'][game_day_t][tables["home"][game_day_t]['team_id'] == t_id]['form_for'].values[0]) if home else calc_form(tables['away'][game_day_t][tables["away"][game_day_t]['team_id'] == t_id]['form_for'].values[0])
    table_dict['opp_home_away_form_for'] = calc_form(tables['away'][game_day_o][tables["away"][game_day_o]['team_id'] == o_id]['form_for'].values[0]) if home else calc_form(tables['home'][game_day_o][tables["home"][game_day_o]['team_id'] == o_id]['form_for'].values[0])
    table_dict['team_home_away_form_against'] = calc_form(tables['home'][game_day_t][tables["home"][game_day_t]['team_id'] == t_id]['form_against'].values[0]) if home else calc_form(tables['away'][game_day_t][tables["away"][game_day_t]['team_id'] == t_id]['form_against'].values[0])
    table_dict['opp_home_away_form_against'] = calc_form(tables['away'][game_day_o][tables["away"][game_day_o]['team_id'] == o_id]['form_against'].values[0]) if home else calc_form(tables['home'][game_day_o][tables["home"][game_day_o]['team_id'] == o_id]['form_against'].values[0])
    return table_dict

In [ ]:
ws_ids = [17, 18, 19, 20, 21]
loader_ids = ["1718", "1819", "1920", "2021", "2122"]
league_name = "GER-Bundesliga"
for ws_id, loader_id in zip(ws_ids, loader_ids):
    ws = sd.WhoScored(
        leagues=league_name,
        seasons=[ws_id],
        no_cache=False,
        no_store=False,
        data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
        path_to_browser="/usr/bin/chromium",
        headless=True,
    )
    corona_games = [1376730, 1376707, 1376732, 1376726, 1376733, 1376725, 1376729, 1376716, 1376735, 1376734, 1376721, 1376722, 1376723, 1376728, 1376724, 1376731, 1376727, 1376720, 1376718, 1376717, 1376719]

    loader = ws.read_events(output_fmt='loader', skip=corona_games)
    df_games = loader.games(competition_id=league_name, season_id=loader_id)
    ce = sd.ClubElo(
        no_cache=False,
        no_store=False,
        data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    )


    def make_table_entry(team, score_f, score_a, table_list, empty_table):
        made_entry = False
        for table_id in range(len(table_list)):
            if team not in table_list[table_id]['team_id'].values:
                try:
                    # get data from prev day
                    old = table_list[table_id - 1][table_list[table_id - 1]['team_id'] == team]
                    points = old['points'].values[0]
                    points = points + (3 if score_f > score_a else 1 if score_f == score_a else 0)
                    games = old['games'].values[0] + 1
                    goals_f = old["goals_f"].values[0] + score_f
                    goals_a = old["goals_a"].values[0] + score_a
                    old["form_for"].values[0].append(str(score_f))
                    old["form_against"].values[0].append(str(score_a))
                    form_for = old["form_for"].values[0][-5:]
                    form_against = old["form_against"].values[0][-5:]
                    table_list[table_id] = pd.concat([table_list[table_id], pd.DataFrame({"team_id": [team], "games": [games], "points": [points], "goals_f": [goals_f], "goals_a": [goals_a], "goal_diff": [goals_f - goals_a], "form_for": [form_for], "form_against": [form_against]})], ignore_index=True)
                    made_entry = True
                    break
                except:
                    # first run
                    form_for = [str(score_f)]
                    form_against = [str(score_a)]
                    points = 3 if score_f > score_a else 1 if score_f == score_a else 0
                    table_list[-1] = pd.concat([table_list[-1], pd.DataFrame({"team_id": [team], "games": [1], "points": [points], "goals_f": [score_f], "goals_a": [score_a], "goal_diff": [score_f - score_a], "form_for": [form_for], "form_against": [form_against]})], ignore_index=True)
                    made_entry = True
                    break
            else:
                continue
                
        if not made_entry:        
            table_list.append(empty_table)
            old = table_list[-2][table_list[-2]['team_id'] == team]
            games = old['games'].values[0] + 1
            points = old['points'].values[0]
            points = points + (3 if score_f > score_a else 1 if score_f == score_a else 0)
            goals_f = old["goals_f"].values[0] + score_f
            goals_a = old["goals_a"].values[0] + score_a
            old["form_for"].values[0].append(str(score_f))
            old["form_against"].values[0].append(str(score_a))
            form_for = old["form_for"].values[0][-5:]
            form_against = old["form_against"].values[0][-5:]
            table_list[-1] = pd.concat([table_list[-1], pd.DataFrame({"team_id": [team], "games": [games], "points": [points], "goals_f": [goals_f], "goals_a": [goals_a], "goal_diff": [goals_f - goals_a], "form_for": [form_for], "form_against": [form_against]})], ignore_index=True)

        # return table_list


    empty_table = pd.DataFrame(data=[], columns=["team_id", "games", "points", "goals_f", "goals_a", "goal_diff", "form"])
    tables = {"all": [empty_table], "home": [empty_table], "away": [empty_table]}
    df_games = df_games.sort_values("game_date")
    for g_id in df_games['game_id']:
        h_team = df_games[df_games['game_id'] == g_id]['home_team_id'].values[0]
        a_team = df_games[df_games['game_id'] == g_id]['away_team_id'].values[0]
        h_score = df_games[df_games['game_id'] == g_id]['home_score'].values[0]
        a_score = df_games[df_games['game_id'] == g_id]['away_score'].values[0]

        make_table_entry(h_team, h_score, a_score, tables['all'], empty_table)
        make_table_entry(a_team, a_score, h_score, tables['all'], empty_table)
        make_table_entry(a_team, a_score, h_score, tables['away'], empty_table)
        make_table_entry(h_team, h_score, a_score, tables['home'], empty_table)

        
    for t_id in range(len(tables['all'])):
        tables['all'][t_id].sort_values(['points', "goal_diff"], ascending=False, inplace=True)
        tables["all"][t_id].reset_index(drop=True, inplace=True)
    for t_id in range(len(tables['home'])):
        tables['home'][t_id].sort_values(['points', "goal_diff"], ascending=False, inplace=True)
        tables["home"][t_id].reset_index(drop=True, inplace=True)
    for t_id in range(len(tables['away'])):
        tables['away'][t_id].sort_values(['points', "goal_diff"], ascending=False, inplace=True)
        tables["away"][t_id].reset_index(drop=True, inplace=True)


    # TODO /11 and not actual looking for red cards
    keeper_actions_save = ["keeper_save"]
    # keeper_actions_other = ["keeper_claim", "keeper_punch"] #, "keeper_pick_up"
    defensive_actions = ["tackle", "interception", "clearance"] # "keeper_save"
    count_teams = {"all": {}, "home": {}, "away": {}}
    df_games = df_games.sort_values("game_date")
    for idx, game_id in enumerate(tqdm(df_games['game_id'])):
        # receive dataframes
        df_teams = loader.teams(game_id=game_id)
        up_team_count(count_teams, df_games, game_id)
        # skip first ten games of season because of insufficent data
        # if(count_teams['all'][df_games[df_games['game_id'] == game_id]['home_team_id'].values[0]] < 10 or 
        #     count_teams['all'][df_games[df_games['game_id'] == game_id]['away_team_id'].values[0]] < 10):
        #     continue
        df_players = loader.players(game_id=game_id)
        df_events = loader.events(game_id=game_id)
        df_events.dropna(subset=["player_id"], inplace=True)
        ce_df = ce.read_by_date(np.datetime_as_string(df_games[df_games['game_id'] == game_id]['game_date'].values[0])[:10])
        # defensive actions
        df_actions = spadl.opta.convert_to_actions(df_events, df_games[df_games['game_id'] == game_id].home_team_id.values[0]).merge(spadl.actiontypes_df()).merge(df_players[['player_name', "player_id"]])
        df_actions_ltr = spadl.play_left_to_right(df_actions, df_games[df_games['game_id'] == game_id].home_team_id.values[0])
        df_actions_rtl = spadl.play_left_to_right(df_actions, df_games[df_games['game_id'] == game_id].away_team_id.values[0])
        df_all_defense_pressing = df_actions_ltr[df_actions_ltr['type_name'].isin(defensive_actions) ]
        df_all_defense_pressing = df_all_defense_pressing[df_all_defense_pressing['result_id'] == 1]
        df_all_defense_normal = df_actions_rtl[df_actions_rtl['type_name'].isin(defensive_actions) ]
        df_all_defense_normal = df_all_defense_normal[df_all_defense_normal['result_id'] == 1]
        xt_defense_ratings_pressing = xTModell.rate_defensive(df_all_defense_pressing.reset_index())
        xt_defense_ratings_normal = xTModell.rate_defensive(df_all_defense_normal.reset_index())
        df_xd_press = pd.DataFrame({"time": ((df_all_defense_pressing['period_id']-1)*45) + (df_all_defense_pressing['time_seconds']/60), 
                            "team": df_all_defense_pressing["team_id"], "xd_press": xt_defense_ratings_pressing})
        df_xd_normal = pd.DataFrame({"time": ((df_all_defense_normal['period_id']-1)*45) + (df_all_defense_normal['time_seconds']/60), 
                            "team": df_all_defense_normal["team_id"], "xd_normal": xt_defense_ratings_normal})
        # attacking actions
        df_attacking_actions = xthreat.get_successful_move_actions(df_actions_ltr)
        xt_attacking_ratings = xTModell.rate(df_attacking_actions)
        df_xt = pd.DataFrame({"time": ((df_attacking_actions['period_id']-1)*45) + (df_attacking_actions['time_seconds']/60), 
                            "team": df_attacking_actions["team_id"], "xt": xt_attacking_ratings})
        # goalie actions
        # TODO fix goalie
        # df_keeper_save = df_actions_rtl[df_actions_rtl['type_name'].isin(keeper_actions_save)]
        # df_keeper_save = df_keeper_save[df_keeper_save['result_id'] == 1]
        # xt_keeper_save = xTModell.rate_defensive(df_keeper_save.reset_index())
        # xG actions
        df_shot_actions = df_actions_ltr[df_actions_ltr['type_id'] == 11]
        xG_ratings = xTModell.rate_xG(df_shot_actions.reset_index())
        df_xg = pd.DataFrame({"time": ((df_shot_actions['period_id']-1)*45) + (df_shot_actions['time_seconds']/60), 
                            "team": df_shot_actions["team_id"], "xg": xG_ratings})
        # GI actions 
        df_goals = df_shot_actions[df_shot_actions['result_id'] == 1]
        df_own_goals = df_actions_ltr[df_actions_ltr['result_id'] == 3]
        df_all_goals = pd.concat([df_goals, df_own_goals], ignore_index=True)
        df_all_goals = pd.DataFrame({"time":((df_all_goals['period_id']-1)*45) + (df_all_goals['time_seconds']/60), 
                                    "team": [ x['team_id'] if(x['result_id'] == 1) 
                                                            else df_teams[df_teams["team_id"] != x['team_id']]['team_id'].values[0] 
                                                            for _, x in df_all_goals.iterrows()]})

        # create people dict
        people_dict = {}
        for people_id in df_players['player_id']:
            people_dict[people_id] = {}
            people_dict[people_id]['name'] = df_players[df_players['player_id'] == people_id]['player_name'].values[0]
            people_dict[people_id]['played'] = False if df_players[df_players['player_id'] == people_id]['minutes_played'].values[0] == 0 else True
            people_dict[people_id]['opp_team_name'] = df_teams[df_teams['team_id'] != df_players[df_players['player_id'] == people_id]['team_id'].values[0]]['team_name'].values[0]
            people_dict[people_id]['team_name'] = df_teams[df_teams['team_id'] == df_players[df_players['player_id'] == people_id]['team_id'].values[0]]['team_name'].values[0]
            people_dict[people_id]['team_id'] = df_players[df_players['player_id'] == people_id]['team_id'].values[0]
            people_dict[people_id]['opp_id'] = df_teams[df_teams['team_id'] != people_dict[people_id]['team_id']]['team_id'].values[0]
            people_dict[people_id]['is_starter'] = df_players[df_players['player_id'] == people_id]['is_starter'].values[0]
            people_dict[people_id]['home'] = (df_games[df_games['game_id'] == game_id]['home_team_id'] == people_dict[people_id]['team_id']).values[0]
            people_dict[people_id]['table_info'] = calc_table_pos(count_teams, people_dict[people_id]['team_id'], people_dict[people_id]['opp_id'], people_dict[people_id]['home'])
            people_dict[people_id]['minutes'] = df_players[df_players['player_id'] == people_id]['minutes_played'].values[0]

            people_dict[people_id]['team_elo'] = ce_df.loc[people_dict[people_id]['team_name']].elo
            people_dict[people_id]['opp_elo'] = ce_df.loc[people_dict[people_id]['opp_team_name']].elo
            people_dict[people_id]['league_elo'] = np.mean(ce_df[ce_df['league'] == df_games[df_games['game_id'] == game_id]['competition_id'].values[0]].elo)
            people_dict[people_id]['top_league_elo'] = get_top_league_elo(ce_df)
            
            people_dict[people_id]['full_game_length'] = df_games[df_games['game_id'] == game_id]['duration'].values[0]
            people_dict[people_id]['minutes_started'] = 0 if people_dict[people_id]['is_starter'] else people_dict[people_id]['full_game_length'] - people_dict[people_id]['minutes']
            people_dict[people_id]['minutes_ended'] = people_dict[people_id]['minutes_started'] + people_dict[people_id]['minutes']

            people_dict[people_id]['game_date'] = np.datetime_as_string(df_games[df_games['game_id'] == game_id]['game_date'].values[0])[:10]
            people_dict[people_id]['xG'] = df_xg.loc[(df_xg['team'] == people_dict[people_id]['team_id']) & (df_xg["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_xg["time"] < people_dict[people_id]['minutes_ended'])]["xg"].sum() / 11
            people_dict[people_id]['xT_all'] = df_xt.loc[(df_xt['team'] == people_dict[people_id]['team_id']) & (df_xt["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_xt["time"] < people_dict[people_id]['minutes_ended'])]["xt"].sum() / 11 # all
            people_dict[people_id]['xT_only_pos'] = df_xt.loc[(df_xt['team'] == people_dict[people_id]['team_id']) & (df_xt["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_xt["time"] < people_dict[people_id]['minutes_ended']) & (df_xt['xt'] > 0)]["xt"].sum() / 11# only positiv
            people_dict[people_id]['xD_press'] = df_xd_press.loc[(df_xd_press['team'] == people_dict[people_id]['team_id']) & (df_xd_press["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_xd_press["time"] < people_dict[people_id]['minutes_ended'])]["xd_press"].sum() / 11
            people_dict[people_id]['xD_def'] = df_xd_normal.loc[(df_xd_normal['team'] == people_dict[people_id]['team_id']) & (df_xd_normal["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_xd_normal["time"] < people_dict[people_id]['minutes_ended'])]["xd_normal"].sum() / 11
            # TODO fix goalie
            people_dict[people_id]['xK_save'] = 0
            people_dict[people_id]['xG_against'] = df_xg.loc[(df_xg['team'] == people_dict[people_id]['opp_id']) & (df_xg["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_xg["time"] < people_dict[people_id]['minutes_ended'])]["xg"].sum() / 11
            people_dict[people_id]['xT_against_all'] = df_xt.loc[(df_xt['team'] == people_dict[people_id]['opp_id']) & (df_xt["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_xt["time"] < people_dict[people_id]['minutes_ended'])]["xt"].sum() / 11 # all
            people_dict[people_id]['xT_against_only_pos'] = df_xt.loc[(df_xt['team'] == people_dict[people_id]['opp_id']) & (df_xt["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_xt["time"] < people_dict[people_id]['minutes_ended']) & (df_xt['xt'] > 0)]["xt"].sum() / 11# only positiv
            people_dict[people_id]['gI'] = (df_all_goals.loc[(df_all_goals['team'] == people_dict[people_id]['team_id']) & (df_all_goals["time"] > people_dict[people_id]['minutes_started']) 
                                        & (df_all_goals["time"] < people_dict[people_id]['minutes_ended'])].shape[0] - df_all_goals.loc[(df_all_goals['team'] == people_dict[people_id]['opp_id'])
                                        & (df_all_goals["time"] > people_dict[people_id]['minutes_started']) & (df_all_goals["time"] < people_dict[people_id]['minutes_ended'])].shape[0]) / 11

        # create Proto files
        for player_id in people_dict.keys():
            if not people_dict[player_id]['played']:
                # only handle player if he played
                continue
            # search dir for existing proto file:
            if str(player_id) + ".pb" not in os.listdir("../data/data_0.3/"):
                # create new proto obj
                proto_player = Player()
                proto_player.player_id = player_id
                proto_player.player_name = people_dict[player_id]['name']
                # add new player to csv
                pd.DataFrame({"player_name": [people_dict[player_id]['name']],"id": [player_id]}).to_csv("../data/data_0.3/player_db.csv", mode='a', header=False, index=False, sep=';')
            else:
                proto_player = Player().parse(open(f"../data/data_0.3/{str(player_id)}.pb", "rb").read())
            # create game
            player_game = Game()
            player_game.game_id = game_id
            player_game.game_date = people_dict[player_id]['game_date']
            player_game.xg = people_dict[player_id]['xG']
            player_game.xt_all = people_dict[player_id]['xT_all']
            player_game.xt_only_pos = people_dict[player_id]['xT_only_pos']
            player_game.xd_press = people_dict[player_id]['xD_press']
            player_game.xd_def = people_dict[player_id]['xD_def']
            player_game.xk_save = people_dict[player_id]['xK_save']
            player_game.xg_against = people_dict[player_id]['xG_against']
            player_game.xt_against_all = people_dict[player_id]['xT_against_all']
            player_game.xt_against_only_pos = people_dict[player_id]['xT_against_only_pos']
            player_game.gi = people_dict[player_id]['gI']
            player_game.starter = people_dict[player_id]['is_starter']
            player_game.team = people_dict[player_id]['team_id']
            player_game.home = people_dict[player_id]['home']
            player_game.minutes_played = people_dict[player_id]['minutes']
            player_game.team_elo = people_dict[player_id]['team_elo']
            player_game.opposition_elo = people_dict[player_id]['opp_elo']
            player_game.league_elo = people_dict[player_id]['league_elo'] 
            player_game.top_league_elo = people_dict[player_id]['top_league_elo']

            player_game.team_pos = people_dict[player_id]['table_info']['team_table_pos']
            player_game.opp_position = people_dict[player_id]['table_info']['opp_table_pos']
            player_game.team_pos_home_away = people_dict[player_id]['table_info']['team_home_away_table_pos']
            player_game.opp_position_home_away = people_dict[player_id]['table_info']['opp_home_away_table_pos']
            player_game.team_form_for = people_dict[player_id]['table_info']['team_form_for'] 
            player_game.team_form_against = people_dict[player_id]['table_info']['team_form_against'] 
            player_game.opp_form_for = people_dict[player_id]['table_info']['opp_form_for']
            player_game.opp_form_against = people_dict[player_id]['table_info']['opp_form_against']
            player_game.team_form_home_away_for = people_dict[player_id]['table_info']['team_home_away_form_for']
            player_game.team_form_home_away_against = people_dict[player_id]['table_info']['team_home_away_form_against']
            player_game.opp_form_home_away_for = people_dict[player_id]['table_info']['opp_home_away_form_for'] 
            player_game.opp_form_home_away_against = people_dict[player_id]['table_info']['opp_home_away_form_against'] 
            
            proto_player.expected_game_impact.append(player_game)

            with open(f"../data/data_0.3/{str(player_id)}.pb", "wb") as f:
                f.write(bytes(proto_player))


In [ ]:
# data 0.3
# ita_1 17, 18, 19, 20, 21
# fra_1 17, 18, 19, 20, 21